In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
import numpy as np

<h1 align = center> Data Loading and Preview </h1>

In [4]:
data = pd.read_csv('heart_failure.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                299 non-null    int64  
 1   age                       299 non-null    float64
 2   anaemia                   299 non-null    object 
 3   creatinine_phosphokinase  299 non-null    int64  
 4   diabetes                  299 non-null    object 
 5   ejection_fraction         299 non-null    int64  
 6   high_blood_pressure       299 non-null    object 
 7   platelets                 299 non-null    float64
 8   serum_creatinine          299 non-null    float64
 9   serum_sodium              299 non-null    int64  
 10  sex                       299 non-null    object 
 11  smoking                   299 non-null    object 
 12  time                      299 non-null    int64  
 13  DEATH_EVENT               299 non-null    int64  
 14  death_even

In [5]:
data.head()

,Unnamed: 0,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT,death_event
0,0,75.0,no,582,no,20,yes,265000.00,1.9,130,yes,no,4,1,yes
1,1,55.0,no,7861,no,38,no,263358.03,1.1,136,yes,no,6,1,yes
2,2,65.0,no,146,no,20,no,162000.00,1.3,129,yes,yes,7,1,yes
3,3,50.0,yes,111,no,20,no,210000.00,1.9,137,yes,no,7,1,yes
4,4,65.0,yes,160,yes,20,no,327000.00,2.7,116,no,no,8,1,yes


In [7]:
print('Classes and number of values in the dataset {}'.format(Counter(data.death_event)))

Classes and number of values in the dataset Counter({'no': 203, 'yes': 96})


In [12]:
y = data['death_event']
x = data[['age','anaemia','creatinine_phosphokinase','diabetes','ejection_fraction','high_blood_pressure','platelets','serum_creatinine','serum_sodium','sex','smoking','time']]

<h1 align = center> Data preprocessing </h1>

<h4><p align =center> Converting the categorical features in the DataFrame instance x to one-hot encoding vectors and assign the result back to variable. </p></h4>

In [14]:
x = pd.get_dummies(x)

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 7)

### Data Normalization

In [20]:
ct = ColumnTransformer([('StandardScale', 
                         StandardScaler(), 
                         ['age','creatinine_phosphokinase','ejection_fraction','platelets','serum_creatinine','serum_sodium','time'])],
                         remainder = 'passthrough')

In [21]:
x_train = ct.fit_transform(x_train)
x_test = ct.transform(x_test)

<h1 align = center> Preparing Labels for Classification </h1>

In [24]:
le = LabelEncoder()
y_train = le.fit_transform(y_train.astype(str))
y_test = le.transform(y_test.astype(str))

In [25]:
y_train = to_categorical(y_train, dtype = 'int64')
y_test = to_categorical(y_test, dtype = 'int64')

<h1 align = center> Model Design </h1>

In [36]:
model = Sequential()
model.add(InputLayer(x_train.shape[1],))
model.add(Dense(12, activation = 'relu'))
model.add(Dense(2, activation= 'softmax'))

model.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer = 'adam')


model.fit(x_train, y_train, verbose = 1, epochs = 100, batch_size = 16)

Epoch 1/100
14/14 [==============================] - 3s 5ms/step - loss: 0.8568 - accuracy: 0.4131
Epoch 2/100
14/14 [==============================] - 0s 2ms/step - loss: 0.8045 - accuracy: 0.3662
Epoch 3/100
14/14 [==============================] - 0s 2ms/step - loss: 0.7475 - accuracy: 0.5038
Epoch 4/100
14/14 [==============================] - 0s 2ms/step - loss: 0.6866 - accuracy: 0.5578
Epoch 5/100
14/14 [==============================] - 0s 2ms/step - loss: 0.6349 - accuracy: 0.6365
Epoch 6/100
14/14 [==============================] - 0s 2ms/step - loss: 0.5926 - accuracy: 0.7289
Epoch 7/100
14/14 [==============================] - 0s 2ms/step - loss: 0.5653 - accuracy: 0.7297
Epoch 8/100
14/14 [==============================] - 0s 2ms/step - loss: 0.5774 - accuracy: 0.7134
Epoch 9/100
14/14 [==============================] - 0s 2ms/step - loss: 0.5564 - accuracy: 0.7094
Epoch 10/100
14/14 [==============================] - 0s 2ms/step - loss: 0.5530 - accuracy: 0.7296
Epoch 11/

<h1 align = center> Model Evaluation </h1>

In [41]:
loss, accuracy  = model.evaluate(x_test, y_test)
print('Model Loss: {} \nModel Accuracy: {}'.format(loss, accuracy))

3/3 [==============================] - 0s 4ms/step - loss: 0.3381 - accuracy: 0.8444
Model Loss: 0.3380567729473114 
Model Accuracy: 0.8444444537162781


<h1 align = center> Generating classification report </h1>

In [44]:
y_estimate = model.predict(x_test)

y_estimate = np.argmax(y_estimate, axis = 1)
y_true = np.argmax(y_test, axis = 1)

print(classification_report(y_true, y_estimate))

              precision    recall  f1-score   support

           0       0.87      0.92      0.90        65
           1       0.76      0.64      0.70        25

    accuracy                           0.84        90
   macro avg       0.82      0.78      0.80        90
weighted avg       0.84      0.84      0.84        90

